In [19]:
import os as os

import numpy as np

import tensorflow.keras as keras
import tensorflow.train as train
import tensorflow.nn as nn

from sklearn.model_selection import train_test_split

In [20]:
def convertLabel(aminoAcid): #gives a nb to the aminoacid
    aminoacidList = ["ALA", "ARG", "ASN", "ASP", "CYS", "GLU", "GLN", "GLY", "HIS", "ILE",
                    "LEU", "LYS", "MET", "PHE", "PRO", "PYL", "SEL", "SER", "THR", "TRP", "TYR", "VAL"]
    for i in range (0, 22):
        if aminoAcid == aminoacidList[i]:
            return i

In [21]:
filename = 'matrices1.csv'
def load_data(filename):


    labelList = []

    with open(filename) as f:

        nbOfSamples = int(f.readline())
        i = 0

        shapes = f.readline() # length of sequence + nb of features
        input_shape = int(shapes.split(",")[1])

        # inputs
        distancesList = np.zeros((nbOfSamples, input_shape))
        previousResiduesList = np.zeros((nbOfSamples, 6))
        
        # outputs
        labelList = np.zeros(nbOfSamples)
        


        while i < nbOfSamples:
            # Extracting labels
            sequence = f.readline().split(",")
            labelList[i] = convertLabel(sequence[7])
            previousResiduesList[i, :] = np.array([convertLabel(sequence[i]) for i in range(0, 6)])

            # Extracting distances
            distancesString = f.readline()
            distancesArray = np.array([float(s) for s in distancesString.split(",")])
            distancesList[i, :] = distancesArray

            # empty line
            f.readline()

            i += 1
    return input_shape, distancesList, previousResiduesList, labelList

In [22]:
def create_model():
    # activation function: leaky ReLU
    leakyrelu = lambda x: keras.activations.relu(x, alpha=0.01, max_value=None, threshold=0)

    # distances branch
    distancesBranch = keras.models.Sequential([
        keras.layers.Dense(units = 105, activation = leakyrelu, input_shape = (105,)),
        #keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(units = 210, activation = leakyrelu),
        #keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(units = 210, activation = leakyrelu),
        #keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(units = 100, activation = leakyrelu),
       # keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(units = 75, activation = leakyrelu),
        #keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(units = 53, activation = leakyrelu),
        #keras.layers.Dropout(rate=0.2)
    ])
    
    # residues branch   
    residuesBranch = keras.models.Sequential([
        keras.layers.Dense(units = 6, activation = leakyrelu, input_shape = (6,))
    ])
    
    # concatenation
    combinedInput = keras.layers.concatenate([distancesBranch.output, residuesBranch.output])

    x = keras.layers.Dense(units = 30, activation = leakyrelu, input_shape = (59,))(combinedInput)
    x = keras.layers.Dense(units = 22, activation = nn.softmax)(x)
    
    model = keras.models.Model(inputs = [distancesBranch.input, residuesBranch.input], outputs = x)

    model.compile(optimizer=keras.optimizers.Adam(),
                  loss=keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy'])

    return model

In [23]:
# Data processing
input_shape, distances, previousResidues, output = load_data(filename)


distances_train, distances_test, residues_train, residues_test, y_train, y_test = train_test_split(
    distances, previousResidues, output, test_size=0.2, shuffle = False, random_state=2)

In [24]:
# Create checkppoints

checkpoint_path = "checkpoints/checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(
    checkpoint_path, save_weights_only=True, verbose=1,
    # save weights, every 5-epoch
    period=5)

latest = train.latest_checkpoint(checkpoint_dir)

In [25]:
# create a model instance

model = create_model()

if latest: model.load_weights(latest)

In [26]:
# training
model.fit([distances_train, residues_train], y_train, epochs = 170, callbacks = [cp_callback]) # save checkpoints at the end of each epoch

model.save('my_model.h5')

a, b = model.evaluate([distances_test, residues_test], y_test)
print(a, b)

Epoch 1/170
2120/2120 [==============================] - 1s 557us/step - loss: 3.1581 - acc: 0.0901
Epoch 2/170
2120/2120 [==============================] - 0s 173us/step - loss: 2.8914 - acc: 0.1160
Epoch 3/170
2120/2120 [==============================] - 0s 139us/step - loss: 2.7746 - acc: 0.1495
Epoch 4/170
2120/2120 [==============================] - 0s 142us/step - loss: 2.7238 - acc: 0.1590
Epoch 5/170
1760/2120 [=======================>......] - ETA: 0s - loss: 2.6719 - acc: 0.1682
Epoch 00005: saving model to checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
2120/2120 [==============================] - 1s 244us/step - loss: 2.6626 - acc: 0.1792
Epoch 6/170
2120/2120 [==============================] - 0s 143us/step - loss: 2.6079 - acc: 0.1953
Epoch 7/170
2120/2120 [==============================] - 0s 138us/step - loss: 2.5447 - acc: 0.2127
Epoch 8/170
2120/2120 [==============================] - 0s 151us/step - loss: 2.4615 - acc: 0.2340
Epoch

2120/2120 [==============================] - 0s 208us/step - loss: 0.6163 - acc: 0.8222
Epoch 41/170
2120/2120 [==============================] - 0s 201us/step - loss: 0.5453 - acc: 0.8453
Epoch 42/170
2120/2120 [==============================] - 1s 266us/step - loss: 0.5720 - acc: 0.8377
Epoch 43/170
2120/2120 [==============================] - 1s 258us/step - loss: 0.8267 - acc: 0.7594
Epoch 44/170
2120/2120 [==============================] - 0s 170us/step - loss: 0.6562 - acc: 0.8061
Epoch 45/170
2112/2120 [============================>.] - ETA: 0s - loss: 0.6230 - acc: 0.8163
Epoch 00045: saving model to checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
2120/2120 [==============================] - 1s 236us/step - loss: 0.6228 - acc: 0.8165
Epoch 46/170
2120/2120 [==============================] - 0s 226us/step - loss: 0.5192 - acc: 0.8542
Epoch 47/170
2120/2120 [==============================] - 1s 253us/step - loss: 0.4529 - acc: 0.8792
Epoch 48/1

2120/2120 [==============================] - 0s 187us/step - loss: 0.1070 - acc: 0.9722
Epoch 81/170
2120/2120 [==============================] - 0s 185us/step - loss: 0.0893 - acc: 0.9849
Epoch 82/170
2120/2120 [==============================] - 0s 171us/step - loss: 0.0659 - acc: 0.9877
Epoch 83/170
2120/2120 [==============================] - 0s 147us/step - loss: 0.2756 - acc: 0.9165
Epoch 84/170
2120/2120 [==============================] - 0s 144us/step - loss: 0.2271 - acc: 0.9368
Epoch 85/170
1984/2120 [===========================>..] - ETA: 0s - loss: 0.1456 - acc: 0.9647
Epoch 00085: saving model to checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
2120/2120 [==============================] - 0s 212us/step - loss: 0.1436 - acc: 0.9651
Epoch 86/170
2120/2120 [==============================] - 0s 203us/step - loss: 0.1444 - acc: 0.9646
Epoch 87/170
2120/2120 [==============================] - 0s 187us/step - loss: 0.0922 - acc: 0.9778
Epoch 88/1

2120/2120 [==============================] - 0s 183us/step - loss: 0.0096 - acc: 1.0000
Epoch 121/170
2120/2120 [==============================] - 0s 148us/step - loss: 0.0074 - acc: 1.0000
Epoch 122/170
2120/2120 [==============================] - 0s 116us/step - loss: 0.0071 - acc: 1.0000
Epoch 123/170
2120/2120 [==============================] - 0s 123us/step - loss: 0.0106 - acc: 0.9986
Epoch 124/170
2120/2120 [==============================] - 0s 155us/step - loss: 0.5380 - acc: 0.8708
Epoch 125/170
1920/2120 [==========================>...] - ETA: 0s - loss: 1.9417 - acc: 0.5609
Epoch 00125: saving model to checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
2120/2120 [==============================] - 0s 161us/step - loss: 1.8528 - acc: 0.5755
Epoch 126/170
2120/2120 [==============================] - 1s 291us/step - loss: 0.4607 - acc: 0.8741
Epoch 127/170
2120/2120 [==============================] - 0s 226us/step - loss: 0.2798 - acc: 0.9250
Epo

2120/2120 [==============================] - 0s 175us/step - loss: 1.5124 - acc: 0.6868
Epoch 161/170
2120/2120 [==============================] - 1s 264us/step - loss: 1.4779 - acc: 0.5920
Epoch 162/170
2120/2120 [==============================] - 0s 228us/step - loss: 0.6352 - acc: 0.8104
Epoch 163/170
2120/2120 [==============================] - 0s 168us/step - loss: 0.2761 - acc: 0.9292
Epoch 164/170
2120/2120 [==============================] - 0s 184us/step - loss: 0.2674 - acc: 0.9245
Epoch 165/170
1888/2120 [=========================>....] - ETA: 0s - loss: 0.1604 - acc: 0.9597- ETA: 0s - loss: 0.1639 - acc: 0.
Epoch 00165: saving model to checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
2120/2120 [==============================] - 0s 188us/step - loss: 0.1539 - acc: 0.9613
Epoch 166/170
2120/2120 [==============================] - 0s 160us/step - loss: 0.1213 - acc: 0.9717
Epoch 167/170
2120/2120 [==============================] - 0s 168us/ste

In [27]:
filename1 = "matrices.csv"
input_shape1, distances1, previousResidues1, output1 = load_data(filename1)

distances_train1, distances_test1, residues_train1, residues_test1, y_train1, y_test1 = train_test_split(
    distances1, previousResidues1, output1, test_size=0.2, shuffle = True, random_state=42)

In [28]:
a1, b1 = model.evaluate([distances1, previousResidues1], output1)
print(a1, b1)

1221/1221 [==============================] - 0s 68us/step
0.3791608504835062 0.8886158886647049
